In [ ]:
#%pip install pyarrow
#%pip install -q google-generativeai
#%pip install python-dotenv
#%pip install plotly
%pip install nbformat --upgrade

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv() 

API_KEY = os.getenv("API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [3]:

BASE_URL = f'https://v6.exchangerate-api.com/v6/{API_KEY}'
DATA = '2025/08/25' # Substitua pela data que você quiser

URL = f'{BASE_URL}/latest/USD'

try:
    response = requests.get(URL)
    response.raise_for_status()  # Isso vai levantar um erro para códigos de status HTTP 4xx/5xx

    data = response.json()
    print(data)

except requests.exceptions.HTTPError as err:
    print(f"Ocorreu um erro HTTP: {err}")
except Exception as err:
    print(f"Ocorreu um erro: {err}")

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1757462401, 'time_last_update_utc': 'Wed, 10 Sep 2025 00:00:01 +0000', 'time_next_update_unix': 1757548801, 'time_next_update_utc': 'Thu, 11 Sep 2025 00:00:01 +0000', 'base_code': 'USD', 'conversion_rates': {'USD': 1, 'AED': 3.6725, 'AFN': 68.3283, 'ALL': 82.7218, 'AMD': 382.6216, 'ANG': 1.79, 'AOA': 920.8606, 'ARS': 1418.67, 'AUD': 1.5176, 'AWG': 1.79, 'AZN': 1.7018, 'BAM': 1.6681, 'BBD': 2.0, 'BDT': 121.7538, 'BGN': 1.6678, 'BHD': 0.376, 'BIF': 2970.5923, 'BMD': 1.0, 'BND': 1.2822, 'BOB': 6.9374, 'BRL': 5.4212, 'BSD': 1.0, 'BTN': 88.2145, 'BWP': 13.7197, 'BYN': 3.234, 'BZD': 2.0, 'CAD': 1.3833, 'CDF': 2878.4332, 'CHF': 0.7964, 'CLP': 971.3739, 'CNY': 7.1241, 'COP': 3950.6662, 'CRC': 505.3626, 'CUP': 24.0, 'CVE': 94.0434, 'CZK': 20.7587, 'DJF': 177.721, 'DKK': 6.3674, 'DOP': 63.667, 'DZD': 129.7769, 'EGP': 48.0013, 'ERN': 15

In [4]:
# Salvar o retorno como arquivo JSON na pasta bronze, particionando por timestamp
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d")
output_dir = f"bronze/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/retorno_api_{timestamp}.json", "w", encoding="utf-8") as f:
    import json
    json.dump(data, f, ensure_ascii=False, indent=4)

In [5]:
# Ler o JSON salvo e salvar como Parquet na pasta silver
import pandas as pd
output_dir = f"bronze/{timestamp}"

with open(f"{output_dir}/retorno_api_{timestamp}.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Normaliza o JSON para DataFrame
df = pd.json_normalize(json_data['conversion_rates'])

silver_dir = f"silver/{timestamp}"
os.makedirs(silver_dir, exist_ok=True)
df.to_parquet(f"{silver_dir}/retorno_api_{timestamp}.parquet", index=False)

In [9]:
#print(f"O valor do dólar sobre o euro é: {df['EUR'].values[0]}")

# Carregar o DataFrame diretamente do arquivo Parquet salvo na pasta silver
silver_dir = f"silver/{timestamp}"
df = pd.read_parquet(f"{silver_dir}/retorno_api_{timestamp}.parquet")

# Filtrar e mostrar apenas as moedas da América do Sul
south_american_currencies = ['USD', 'ARS', 'BOB', 'BRL', 'CLP', 'COP', 'PEN', 'UYU', 'VES', 'GYD', 'PYG', 'SRD']
df_south_america = df[south_american_currencies]
# ...restante do código...

#salvando o DataFrame filtrado como parquet na pasta gold

gold_dir = f"gold/{timestamp}"
os.makedirs(gold_dir, exist_ok=True)
df_south_america.to_parquet(f"{gold_dir}/south_american_currencies.parquet", index=False)   

In [2]:
#Consollidano os resultado para um dataframe final 

gold_base = "gold"
consolidated = []

for folder in sorted(os.listdir(gold_base)):
    folder_path = os.path.join(gold_base, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".parquet"):
                df = pd.read_parquet(os.path.join(folder_path, file))
                # Se não existe coluna USD, adiciona com valor 1
                if "USD" not in df.columns:
                    df["USD"] = 1
                # Adiciona coluna com o nome da pasta (timestamp)
                df["timestamp"] = folder
                consolidated.append(df)

# Junta todos os DataFrames
df_consolidated = pd.concat(consolidated, ignore_index=True)

# Reorganiza para ter timestamp como linha e moedas como colunas
df_final = df_consolidated.set_index("timestamp")

# Exibe o DataFrame consolidado
df_final

,ARS,BOB,BRL,CLP,COP,PEN,UYU,VES,GYD,PYG,SRD,USD
timestamp,,,,,,,,,,,,
20250904,1360.17,6.9203,5.4534,973.4898,4004.8407,3.5341,40.0037,151.7627,209.0719,7228.9684,38.6265,1
20250909,1422.17,6.9105,5.4139,965.7960,3964.2089,3.5160,40.0851,154.9825,209.1806,7218.2281,38.9228,1
20250910,1418.67,6.9374,5.4212,971.3739,3950.6662,3.5073,40.0394,156.3739,209.1599,7159.5773,39.0245,1


In [4]:
import plotly.graph_objects as go
import plotly.io as pio

# df_final: DataFrame consolidado, index=timestamp, columns=moedas
df_final = df_final.sort_index()  # Garante ordem cronológica

# Lista de moedas disponíveis (exceto USD)
moedas = [col for col in df_final.columns if col != "USD"]

# Cria o gráfico sem o USD
fig = go.Figure()

# Adiciona as moedas como traces, inicialmente invisíveis
for moeda in moedas:
    fig.add_trace(go.Scatter(
        x=df_final.index,
        y=df_final[moeda],
        mode="lines+markers",
        name=moeda,
        visible=False
    ))

# Botões para selecionar moedas
buttons = []
for i, moeda in enumerate(moedas):
    visible = [j == i for j in range(len(moedas))]
    buttons.append(dict(
        label=moeda,
        method="update",
        args=[{"visible": visible},
              {"title": f"Variação da moeda {moeda}"}]
    ))

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        x=1.15,
        y=1,
        showactive=True,
        buttons=buttons
    )],
    title="Variação das moedas da América do Sul",
    xaxis_title="Timestamp",
    yaxis_title="Valor em relação ao USD",
    legend_title="Moeda",
    height=600
)

fig.show()

In [ ]:
#Conecta Gemini 

import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-1.5-flash")

def gerar_insights_gemini(df, n_linhas=10):
    """
    Gera insights usando Google Gemini Free API
    :param df: DataFrame a ser analisado
    :param n_linhas: Número de linhas do dataset para enviar ao modelo
    """
    sample = df.head(n_linhas).to_string()
    prompt = f"""
    Você é um analista de dados. Analise o seguinte dataset de moedas da América do Sul em relação ao dólar:

    {sample}

    Gere insights claros e objetivos:
    - Quais moedas estão mais valorizadas e mais desvalorizadas?
    - Quais padrões interessantes podem ser observados?
    - Quais análises adicionais poderiam ser feitas com este dataset?
    - O dólar comparado com o valor do Real está alto. O que pode influenciar esse valor? Isso acompanha os valores das outras moedas da america do sul?
    - Você pode pontuar eventos mais recentes que contribuem para a desvalorização do real em relação ao Dolar? Considerando que a data da cotação como {timestamp }

    """

    response = model.generate_content(prompt)
    return response.text

# Exemplo de uso:
insights = gerar_insights_gemini(df_south_america)
print("=== Insights gerados pelo Gemini ===")
print(insights)


/home/adn/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== Insights gerados pelo Gemini ===
Análise do Dataset de Moedas da América do Sul em relação ao Dólar (2025-09-10)

O dataset apresenta as taxas de câmbio de diversas moedas sul-americanas em relação ao dólar americano em uma data específica (2025-09-10).  A análise se baseia nos dados fornecidos, limitados a uma única data, o que impede conclusões definitivas sobre tendências a longo prazo.

**1. Moedas Mais e Menos Valorizadas:**

* **Mais Valorizadas (Menos unidades da moeda local por 1 dólar):**  PEN (Sol Peruano) e BOB (Boliviano) apresentam as menores taxas de câmbio, indicando maior valorização em relação ao dólar.

* **Menos Valorizadas (Mais unidades da moeda local por 1 dólar):**  VES (Bolívar Soberano Venezuelano), seguido de ARS (Peso Argentino) e PYG (Guarani Paraguaio) mostram as maiores taxas de câmbio, indicando significativa desvalorização frente ao dólar.

**2. Padrões Interessantes:**

* **Grande Variabilidade:** Observa-se uma grande dispersão nos valores das taxa